In [1]:
# Replace api key with .env file for privacy
import os
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv('NASDAQ_API_KEY')

In [2]:
# Import relevant module
import requests
import statistics

In [3]:
# Used Nasdaq API to pull out a small sample of the data (only one day) as example
r = requests.get("https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-01-02&api_Key=auth", auth=('user', API_KEY))

# Checked if status code successfull (200)
r.status_code

200

In [4]:
# Inspection of JSON structure object created
r.json()

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2020-12-01T14:48:09.907Z',
  'newest_available_date': '2020-12-01',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2017-01-01',
  'end_date': '2017-01-02',
  'data': [['2017-01-02',
    34.99,
    35.94,
    34.99,
    35.8,
    None,
    44700.0,
    1590561.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

# API Tasks

#### 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X as json, for entire year of 2017 (date format is YYYY-MM-DD).

In [5]:
r = requests.get("https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2018-01-01&api_Key=auth", auth=('user', API_KEY))

# Checked status code was successful (200)
r.status_code

200

#### 2. Converted the returned JSON object into a Python dictionary and verified keys.

In [6]:
afx = dict(r.json())

# Update dictionary for easier use and view
afx = afx['dataset']

print(afx.keys())

dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])


In [7]:
# Separated column names for identification of data
column_names = afx['column_names']
print(column_names)

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [8]:
# Separated all data for identification by each day
data = afx['data']

In [9]:
# Separated days for identification by each date
all_days = [day[0] for day in data]

In [10]:
# Created tuples for all data according to column_names
afx_tup = [dict(zip(column_names, dat)) for dat in data]

# Created dictionary to reference data dictionaries by date
afx_2017 = dict(zip(all_days, afx_tup))

#### 3. Calculated what the highest and lowest opening prices were for the stock in period of year.

In [11]:
# Indexed the "Open" key to get values for prices at open without prices listed as None
high_low_open = [day['Open'] for date, day in afx_2017.items() if day['Open'] != None]

# Sorted the indexed list based on descending order
high_low_open.sort(reverse=True)

# Popped the highest price at open per the year by index 
high_open = high_low_open.pop(0)

# Popped the lowest price at open per the year by reverse indexing
low_open = high_low_open.pop(-1)

print("The highest and lowest opening prices were {}(Highest) and {}(Lowest) for the price of AFX_X's stock in the year 2017.".format(high_open, low_open))

The highest and lowest opening prices were 53.11(Highest) and 34.0(Lowest) for the price of AFX_X's stock in the year 2017.


#### 4. What was the largest change in any one day (based on High and Low price).

In [12]:
# Created a price difference of High to low based on a list comprehension for 2017
daily_change = [abs(round((day['Low'])-day['High'], 2)) for date, day in afx_2017.items()]

# Used a list comprehension to show percentages of daily changes
percent_changes = [round(change * 100, 1) for change in daily_change]

# Narrowed down the changes by the largest change price and percentage
largest_change_percent = (max(daily_change), max(percent_changes))

print("The largest change in any one day (based on High and Low price) was the price of {} at price difference of at least {}%.".format(
    largest_change_percent[0],
    largest_change_percent[1]))

The largest change in any one day (based on High and Low price) was the price of 2.81 at price difference of at least 281.0%.


#### 5. The largest change between any two days (based on Closing Price).

In [13]:
# Created a price difference of High to low based on a list comprehension for 2017
closing_prc = [day['Close'] for date, day in afx_2017.items()]

# Used the max and min function to get prices at Close
max_cp = max(closing_prc)
min_cp = min(closing_prc)

# Max and Min positions for referencing dates with indexes
# max_pos = [(date, day['Close']) for date, day in afx_2017.items() if day['Close'] == max_cp]
# min_pos = [(date, day['Close']) for date, day in afx_2017.items() if day['Close'] == min_cp]

print("The largest change or range between any two days (based on Closing Price) is the highest price of {} and the lowest price of {} equaling a difference of {}."
      .format(
          max_cp,
          min_cp,
          max_cp-min_cp
      ))

The largest change or range between any two days (based on Closing Price) is the highest price of 53.09 and the lowest price of 34.06 equaling a difference of 19.03.


#### 6. The average daily trading volume during the year 2017.

In [14]:
# List comprehesion for trading volume during 2017
daily_tv = [day['Traded Volume'] for date, day in afx_2017.items()]

# Sum of all days based on AFX_X's "Traded Volume"
total_dtv = sum(daily_tv)

# Length of all trading days for the year 2017 based on AFX_X's "Traded Volume"
trading_days = len(daily_tv)

# Mean value of all trading days for the year 2017 based on AFX_X's "Traded Volume"
avr_volume_2017 = total_dtv / trading_days

print("The average daily trading volume during the year 2017 is %11111.2f." % avr_volume_2017)

The average daily trading volume during the year 2017 is                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

#### 7. (Optional) What was the median trading volume during the year 2017. (Note: you may need to implement your own function for calculating the median.)

In [15]:
# Used the built-in Python standard module to calculate median trading volume during 2017
median_tv = statistics.median(daily_tv)

print("The median trading volume during the year 2017 is %11111.2f." % median_tv)

The median trading volume during the year 2017 is                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       